# Tutorial 2: Annotate cell types in single-cell RNA-seq data

Welcome to this tutorial on annotating single-cell datasets with reference collections. The **scRNAseq** ([R/Bioc](https://bioconductor.org/packages/devel/data/experiment/html/scRNAseq.html), [Python](https://github.com/BiocPy/scrnaseq)) package provides access to public single-cell RNA-seq datasets for use by other Bioconductor/BiocPy packages and workflows. These datasets are stored in language-agnostic representations described in [ArtifactDB](https://github.com/artifactdb), enabling easy access to datasets and analysis results across multiple programming languages such as R and Python. We will showcase how to integrate and process single-cell datasets across languages, such as R and Python, and how to annotate cell types using reference datasets.

## Outline

In this tutorial, we'll walk through how to:

1. Explore the `scrnaseq` package and access public single-cell RNA-seq datasets.
2. Perform basic operations on `SingleCellExperiment` objects, the core data structure for single-cell data.
3. Annotate cell types using reference datasets from the `celldex` package.

## Prerequisites

Before we begin, please ensure that you have the following prerequisites installed:

- Python 3.8 or later with dependencies listed [here](https://github.com/BiocPy/BiocWorkshop2024/blob/master/requirements.txt).
- R 4.4.0 and Bioconductor packages listed [here](https://github.com/BiocPy/BiocWorkshop2024/blob/master/rpackages.R).

Install the Python packages using pip:

```sh
pip install scrnaseq celldex singler
```

Install the R packages using BiocManager:

```r
BiocManager::install(c("scRNAseq", "celldex", "SingleR"), 
     repos='http://cran.us.r-project.org')
```

## 1. Accessing and exploring single-cell datasets

Let's explore the `scrnaseq` package and learn how to access public single-cell RNA-seq datasets. Datasets published to the `scrnaseq` package are decorated with metadata such as the study title, species, number of cells, etc., to facilitate discovery. Let's see how we can list and search for datasets.

### 1.1 List all datasets

The `list_datasets()` function in Python or `surveyDatasets()` in R will display all available datasets published to the `scRNAseq` collection along with their metadata. To list all available datasets in the `scrnaseq` package and displays their names, titles, and versions:

In [1]:
import scrnaseq
datasets = scrnaseq.list_datasets()
datasets[["name", "title", "version"]].head(3)

/Users/kancherj/miniforge3/envs/bioc2024/lib/python3.10/site-packages/gypsum_client/fetch_metadata_database.py:126: UserWarning: Failed to check the last modified timestamp: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
  warnings.warn(


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

#### R
```r
suppressMessages(library(scRNAseq))
all_ds <- surveyDatasets()
head(all_ds[, c("name", "title", "version")], 3)
```

### 1.2 Search for datasets

You can also search for datasets based on metadata using `search_datasets()` in Python or `searchDatasets()` in R. This supports both simple text queries and complex boolean expressions.

Let's search for datasets containing the term "pancreas" and displays their names, titles, and versions.

In [ ]:
import scrnaseq

pancreas_datasets = scrnaseq.search_datasets("pancreas")
pancreas_datasets[["name", "title", "version"]].head(3)

#### R
```r
pancreas_ds <- searchDatasets("pancreas")
head(pancreas_ds[, c("name", "title", "version")], 3)
```

#### 1.2.1 Advanced searches

For more complex searches involving boolean operations, use `define_text_query()` in Python or `defineTextQuery()` in R. Here's an example to find datasets using the mouse reference genome (`GRCm38`) and containing the words `neuro` or `pancrea`.

```{tip}
Check out the reference manual for more details and usage of these functions.
```

In [ ]:
from gypsum_client import define_text_query
import scrnaseq

res = scrnaseq.search_datasets(
     define_text_query("GRCm38", field="genome")
     & (
          define_text_query("neuro%", partial=True)
          | define_text_query("pancrea%", partial=True)
     )
)
res[["name", "title", "version"]].head(3)

#### R
```r
suppressWarnings(library(gypsum))
res <- searchDatasets(
    defineTextQuery("GRCm38", field="genome") &
    (defineTextQuery("neuro%", partial=TRUE) | 
     defineTextQuery("pancrea%", partial=TRUE))
)
head(res[,c("name", "title", "version")], 3)
```

This performs a complex search to find datasets tagged as "mouse" in the reference genome field and containing the keywords "neuro" or "pancrea".

```{important}
Once a dataset is identified, always list the name and version of the dataset in your scripts for reproducibility.
```

## 2. Download dataset

After identifying a dataset of interest, use `fetch_dataset()` in Python or `fetchDataset()` in R to download the dataset. This will load the dataset as a `SingleCellExperiment` object.

```{note}
R/Bioconductor users might already be familiar with the [SingleCellExperiment](https://bioconductor.org/packages/release/bioc/html/SingleCellExperiment.html) class. BiocPy also provides similar implementation in the [singlecellexperiment](https://github.com/BiocPy/SingleCellExperiment) package.
```

For this tutorial, let's download the `zeisel-brain` dataset:

In [ ]:
import scrnaseq
sce = scrnaseq.fetch_dataset("zeisel-brain-2015", "2023-12-14")
print(sce)

#### R
```r
sce <- fetchDataset("zeisel-brain-2015", "2023-12-14")
sce
```

### 2.1 Side-quest on `SingleCellExperiment` in Python

The Python implementation of the `SingleCellExperiment` class adheres to Bioconductor's specification and offers similar interface and methods. Our goal is to make it simple for analysts to switch between R and Python.

```{note}
For more details on the design, refer to the [BiocPy developer guide](https://github.com/BiocPy/developer_guide) or the [singlecellexperiment](https://github.com/BiocPy/SingleCellExperiment) documentation.
```

This Python code demonstrates basic operations on a `SingleCellExperiment` object, including retrieving assay names, column names, column metadata, accessing counts, and coercing to an `AnnData` object for interoperability with existing analysis ready eco-systems in Python.

To display assays names from the object:

In [ ]:
print("Assays names: ", sce.get_assay_names()) # or sce.assay_names

To access cell barcodes or ids:

In [ ]:
print("Cell barcodes (first 10): ", sce.get_column_names()[:10]) # or sce.column_names

To access all cell annotations:

In [ ]:
print("Column metadata: ", sce.get_column_data()) # or sce.column_data

To access an assay matrix:

In [ ]:
print("Counts matrix: ", sce.assays["counts"]) # or # sce.assay("counts")

The package uses [delayed arrays](https://github.com/biocpy/delayedarray) (similar to the R/Bioconductor's [DelayedArray](https://www.bioconductor.org/packages/release/bioc/html/DelayedArray.html)), to load file-backed arrays and matrices. This reduces memory usage when loading large datasets. Methods are available to coerce delayed arrays to [sparse matrix representations](https://docs.scipy.org/doc/scipy/reference/sparse.html) from the scipy package:

In [ ]:
from delayedarray import to_scipy_sparse_matrix
print("counts as csr: ")
print(repr(to_scipy_sparse_matrix(sce.assays["counts"], "csc")))

To simplify this, we provide the `realize_assays` option to load matrices fully into memory when fetching the dataset.

In [ ]:
sce = scrnaseq.fetch_dataset(
    "zeisel-brain-2015", "2023-12-14", 
    realize_assays=True)
print(sce)

In addition, we provide coercions from `SingleCellExperiment` class to take advantage of methods in the Python ecosystem, e.g. scverse and AnnData.

In [ ]:
print("coerce to AnnData: ", sce.to_anndata())

## 3. Annotate cell types

We can now annotate cell types by using reference datasets and matching cells based on their expression profiles. In this tutorial, we will use [SingleR](https://github.com/SingleR-inc/SingleR) in R or its Python equivalent [singler](https://github.com/BiocPy/singler).

Before running the `singler` algorithm, we need to download an appropriate reference dataset from the `celldex` package.

### 3.1 Access reference datasets from `celldex`

Similar to the `scRNAseq` package, the `celldex` package provides access to the collection of reference expression datasets with curated cell type labels, for use in procedures like automated annotation of single-cell data or deconvolution of bulk RNA-seq to reference datasets. These datasets are also stored in language-agnostic representations for use in downstream analyses.

For this tutorial, let's download the [Mouse RNA-seq](https://www.immgen.org/) reference from `celldex` using `fetch_reference()` in Python or `fetchReference()` in R. This reference consists of a collection of mouse bulk RNA-seq data sets downloaded from the gene expression omnibus ([Benayoun et al. 2019](https://doi.org/10.1101/gr.240093.118)). A variety of cell types are available, again mostly from blood but also covering several other tissues.

In [ ]:
import celldex

mouse_rnaseq_ref = celldex.fetch_reference(
    "mouse_rnaseq", "2024-02-26", 
    realize_assays=True)
print(mouse_rnaseq_ref)

#### R
```r
suppressWarnings(library(celldex))
mouse_rnaseq_ref <- fetchReference("mouse_rnaseq", "2024-02-26", realize.assays=TRUE)
mouse_rnaseq_ref
```


Now, let's annotate cells from the `zeisel-brain` dataset using the `mouse_rnaseq` reference dataset.

In [ ]:
import singler

matches = singler.annotate_single(
    test_data=sce, 
    ref_data = mouse_rnaseq_ref,
    ref_labels = "label.main"
)

import pandas as pd

pd.Series(matches["best"]).value_counts()

#### R
```r
suppressWarnings(library(SingleR))
cell_labels <- SingleR(test = assay(sce, "counts"), ref = mouse_rnaseq_ref, labels = mouse_rnaseq_ref$label.main)

table(cell_labels$labels)
```

Give this is a brain dataset, the presence of neuron's and other brain-related cell types makes sense.


## 4. Analyze single-cell RNA-seq datasets

![single-cell-methods](../assets/single-cell-space.jpg)

Aaron has implemented the single-cell methods from scran in C++. This allows us to reuse the same implementation in JS and develop applications for analyzing single-cell data ([Kana](https://github.com/kanaverse/kana)), or in Python through the [scranpy](https://github.com/BiocPy/scranpy) package.

To analyze the dataset using the default parameters:

In [ ]:
import scranpy

results = scranpy.analyze_sce(sce)

# results is a complex object, 
# let's explore the umap and tsne dimensions
print(results.tsne)

### 4.1 Want to know whats happening in the entire analysis

Running the `analyze_sce()` function uses the default parameters to run the single-cell workflow. If you want to customize or want to have fine-grained control on the analysis steps, set the parameter `dry_run=True`.

```{note}
This prints out the exact series of steps the function runs under the hood to perform the analysis. You can then use this to customize the analysis to your specific dataset or use case.
```

In [ ]:
print(scranpy.analyze_sce(sce, dry_run=True))

```{tip}
Users can also run individual steps from the analysis without having to perform the full analysis, e.g. compute log-normalized counts or find markers.
```

## 5. Visualize Results

I can't have a tutorial without a section on visualization or figures.

We will use the seaborn and matplotlib packages in Python to create visualizations. We'll plot the t-SNE embedding and color the cells by their cluster assignments.

In [ ]:
import seaborn as sns
sns.scatterplot(
    x=results.tsne.x, y=results.tsne.y, 
    hue=results.clusters, palette="Paired"
)

Now let's color the embedding with the cell types we identified from `celldex`. We ran the singleR algorithm on the full datasets, but scranpy filtered a few cells during the QC step. Let's identify which cells were kept.

In [ ]:
to_keep = [i for i,x in enumerate(results.rna_quality_control_filter) if x == False]
filtered_matches = [matches["best"][i] for i in to_keep]

In [ ]:
import seaborn as sns
sns.scatterplot(
    x=results.tsne.x, y=results.tsne.y, 
    hue=filtered_matches, palette="Paired"
)

Similarly also explore the UMAP embedding:

In [ ]:
import seaborn as sns
sns.scatterplot(
    x=results.umap.x, y=results.umap.y, 
    hue=filtered_matches, palette="Paired"
)

## 6. Exercises

1. Share or upload your datasets to scrna-seq package. Instructions to upload are available in their respective [R/Bioc](https://bioconductor.org/packages/release/data/experiment/html/scRNAseq.html) and [Python](https://github.com/BiocPy/scrnaseq) packages.
2. Explore top markers for each cluster identified by scranpy.
3. Save your results and explore in [Kana](https://github.com/kanaverse/kana).

## Conclusion

Congratulations! You have now completed the tutorial on accessing single-cell datasets using `scRNAseq` and `ArtifactDB`, and annotating cell types using reference datasets from `celldex`. For more detailed usage and advanced analyses, refer to the respective documentation of these packages.